## Introspection

### Introspecting types

#### type() function
* type(object) returns the \_\_class\_\_ attribute of the object
* type(type(object)) returns type class
* class objects store their name on their \_\_name\_\_ attribute

In [76]:
i = 7

# type(i) shows the __class__ attribute of i
print(type(i))

# class objects store their name in __name__ attribute
print(i.__class__.__name__)

# repr(i) show shows __class__ attribute of i
print(repr(i))

# we can directly use the __class__() returned 
# from type to constrcuct new instance
print(type(i)(78))

# type(type(i)) is type class
print(type(type(i)))

<class 'int'>
int
7
78
<class 'type'>


#### issubclass() function
* return boolean value of whether the first augument (must be a class object) is the subclass of the second argument, or any of the second arguments if the second argument is a tuple of class objects

In [77]:
# type is a subclass of object
print(issubclass(type, object))

# the type of object is type
print(type(object))

True
<class 'type'>


#### isinstance() function
* return boolean value of whether the first augument (an object) is an instance of the second argument, or any of the second arguments if the second argument is a tuple of class objects

In [78]:
# i is an instance of int
print(isinstance(i, int))

True


#### Summary
* type checks should be avoided in Python
* is type checks are necessary, prefer isinstanc() and issubclass() over direct comparison of type objects
* objects store their type on \_\_class\_\_ attribute

### Introspecting objects

#### dir(object)
* returns both attributes and methods as callable attributes of the object
* a as a number, has denominatior and numerator attribute to express fraction type
  + it also has image, ral and conjugate attributes for complex numbers

In [79]:
a = 42
print(dir(a))

['__abs__', '__add__', '__and__', '__bool__', '__ceil__', '__class__', '__delattr__', '__dir__', '__divmod__', '__doc__', '__eq__', '__float__', '__floor__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getnewargs__', '__gt__', '__hash__', '__index__', '__init__', '__init_subclass__', '__int__', '__invert__', '__le__', '__lshift__', '__lt__', '__mod__', '__mul__', '__ne__', '__neg__', '__new__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rlshift__', '__rmod__', '__rmul__', '__ror__', '__round__', '__rpow__', '__rrshift__', '__rshift__', '__rsub__', '__rtruediv__', '__rxor__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', '__trunc__', '__xor__', 'as_integer_ratio', 'bit_count', 'bit_length', 'conjugate', 'denominator', 'from_bytes', 'imag', 'numerator', 'real', 'to_bytes']


### getattr(objec, attr)
* we can use this function to access the attribute of an object
* this is the same as directly calling the attribute from the object (object.attr)
* trying to retrieve an attribute that doesn't exist will trigger an AttributeError
* we can check if an attribute exists by hasattr(object, attr) method

In [80]:
print(getattr(a, 'denominator'))
print(a.denominator)

1
1


#### Chaining the errors
* errors can be handled by chaining them to
  + raise appropriate errors
  + provide more detailed information of the cause of the error
* in the following code example
  + AttributeError was raised as the direct result of accessing the non-existing numerator and denominator attributes
  * this AttributeError was then chained to a more appropriate TypeError using the raise...from statement
    + this gives the end users more useful information about the TypeError of the input number, in addition to the detailed information about how this error occured from the AttributeError in the first place

In [81]:
from fractions import Fraction


def mixed_numeral(vulgar):
    try:
        integer = vulgar.numerator // vulgar.denominator
        fraction = Fraction(vulgar.numerator - integer * vulgar.denominator,
                            vulgar.denominator)

        return integer, fraction
    except AttributeError as e:
        raise TypeError("{} is not a rational number".format(vulgar)) from e

In [82]:
# input the correct rational number
mixed_numeral(Fraction('11/10')) 

(1, Fraction(1, 10))

In [83]:
# input the incorrect type of number
mixed_numeral(1.7) 

TypeError: 1.7 is not a rational number

#### Summary
* generally use Easy to ask forgiveness than permission (EAFP) style programming using try/except block rather than hasattr/getattr
* programs using hasattr() can quickly become messay
* the optimistic approach can be faster by using try/except because hasattr() uses try/except internally

### Introspecting Scopes

#### Introspecting the global namespace
* globals()
  + returns a dictionary representing the global namespace
  + furthermore, the dictionary return is, itself, the global namespace
    + we can add variables directly to the returned dictionary object, and use the variables as a global variable

In [84]:
# define a variable to the dictionary returned by globals()
# and use it in the global namespace
globals()['tau'] = 6.283185
tau

6.283185

#### Introspecting local namespace
* locals()
* in the following code example, we defined a local function, report_scope, and print out the local namespace inside it
  + the local namespace contains the arg, the pprint function imported in the function, and the local variable x

In [85]:
def report_scope(arg):                                                      
    from pprint import pprint as pp                                         
    x = 496                                                                 
    pp(locals(), width=10)                                                  
                                                                            
report_scope(42)                      

{'arg': 42,
 'pp': <function pprint at 0x000001B43375E710>,
 'x': 496}


#### Literal String Interpolation
* PEP 498 introduced a new string literal: f-strings
* f-strings interpolate names from namespaces directly into strings 

In [86]:
# use format to fill in variable values in strings
name = "Joe Bloggs"
age = 28                                                                    
country = "New Zealand"                                                     
"{name} is {age} years old and is from {country}".format(**locals()) 

'Joe Bloggs is 28 years old and is from New Zealand'

In [87]:
# use f-string to fill in variable values in strings
name = "Joe Bloggs"
age = 28                                                                    
country = "New Zealand"                                                     
f"{name} is {age} years old and is from {country}"

'Joe Bloggs is 28 years old and is from New Zealand'

### Inspect Module
* contains advanced tools for introspecting Python objects in more details
* code example: batch.py contains the code that we will introspect by inspect module
* the content of batch.py is the following:

```python
from itertools import chain

class Batch:
    def __init__(self, iterables=()):
        self._iterables = list(iterables)

    def append(self, iterable):
        self._iterables.append(iterable)

    def __iter__(self):
        return chain(*self._iterables)
```    

* we can check is an object is a module
* we can get all the member objects of this module
* we can fiter the members returned using inspect's built-in predicates
  + here we filter out only the class objects
* it is surprising class chain is listed in batch module in code example 4. this is because
  + chain() is a class, not a function.
  + it looks like a function because it violates the PEP8 naming conventions
  + calling chain() is actually invoking a constructor
* any class defined in a module or imported into a module is in that module's namespace and available as a member of the module
* as an example, if you import a class from another module, that class will be available in the namespace, as shown in code example 5
* we can obtain details of class methods. 
  + we can get all the class methods using isfunction pedicate in getmembers() function
  + we focus on a specific method, for example, \_\_init\_\_() method (code example 7)
    + we get the signature object from the method using inspect.signature
    + we can get the parameters of the method signature as an OrderedDict
      + further retrieve the items from the OrderedDict 
    + Signature stores information on type annotations (type annotions were introduced in PEP 484), as shown in example code 8    
  + Function implemented in C or other languages may not provide metadata and cause signature() to fail with ValueError  

In [88]:
import inspect
import batch
from pprint import pprint as pp

In [89]:
# code example 1

# check if the object is a module
pp(inspect.ismodule(batch))

True


In [90]:
# code example 2

# check all the members in the module
print(inspect.getmembers(batch))

[('Batch', <class 'batch.Batch'>), ('__builtins__', {'__name__': 'builtins', '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.", '__package__': '', '__loader__': <class '_frozen_importlib.BuiltinImporter'>, '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in'), '__build_class__': <built-in function __build_class__>, '__import__': <built-in function __import__>, 'abs': <built-in function abs>, 'all': <built-in function all>, 'any': <built-in function any>, 'ascii': <built-in function ascii>, 'bin': <built-in function bin>, 'breakpoint': <built-in function breakpoint>, 'callable': <built-in function callable>, 'chr': <built-in function chr>, 'compile': <built-in function compile>, 'delattr': <built-in function delattr>, 'dir': <built-in function dir>, 'divmod': <built-in function divmod>, 'eval': <built-in function eval>, 'exec': <built-in fu

In [91]:
# code example 3

# print all the predicate functions of inspect for filtering members results
# these prediates starts with is and can be used as the second argument to getmembers()
print(dir(inspect))

['ArgInfo', 'ArgSpec', 'Arguments', 'Attribute', 'BlockFinder', 'BoundArguments', 'CORO_CLOSED', 'CORO_CREATED', 'CORO_RUNNING', 'CORO_SUSPENDED', 'CO_ASYNC_GENERATOR', 'CO_COROUTINE', 'CO_GENERATOR', 'CO_ITERABLE_COROUTINE', 'CO_NESTED', 'CO_NEWLOCALS', 'CO_NOFREE', 'CO_OPTIMIZED', 'CO_VARARGS', 'CO_VARKEYWORDS', 'ClassFoundException', 'ClosureVars', 'EndOfBlock', 'FrameInfo', 'FullArgSpec', 'GEN_CLOSED', 'GEN_CREATED', 'GEN_RUNNING', 'GEN_SUSPENDED', 'OrderedDict', 'Parameter', 'Signature', 'TPFLAGS_IS_ABSTRACT', 'Traceback', '_ClassFinder', '_ClassMethodWrapper', '_KEYWORD_ONLY', '_MethodWrapper', '_NonUserDefinedCallables', '_PARAM_NAME_MAPPING', '_POSITIONAL_ONLY', '_POSITIONAL_OR_KEYWORD', '_ParameterKind', '_VAR_KEYWORD', '_VAR_POSITIONAL', '_WrapperDescriptor', '__author__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_check_class', '_check_instance', '_empty', '_filesbymodname', '_findclass', '_finddoc', '_has_code

In [92]:
# code example 4
# note that chain is listed as a member class in batch module
print(inspect.getmembers(batch, inspect.isclass))

[('Batch', <class 'batch.Batch'>), ('chain', <class 'itertools.chain'>)]


In [93]:
# code example 5
from batch import chain
list(chain([1, 2, 3], [4, 5, 6]))

[1, 2, 3, 4, 5, 6]

In [94]:
# code example 6
# get all function members of Batch class
inspect.getmembers(batch.Batch, inspect.isfunction)

[('__init__', <function batch.Batch.__init__(self, iterables=())>),
 ('__iter__', <function batch.Batch.__iter__(self)>),
 ('append', <function batch.Batch.append(self, iterable)>)]

In [95]:
# code example 7
# get signature of a function
init_sig = inspect.signature(batch.Batch.__init__)
print(init_sig)
print(init_sig.parameters)
print(repr(init_sig.parameters['iterables'].default))

(self, iterables=())
OrderedDict([('self', <Parameter "self">), ('iterables', <Parameter "iterables=()">)])
()


In [96]:
# code example 8
# introspecting Type Annotations
import inspect   
# define a function, num_vowels
def num_vowels(text: str) -> int:                                           
    return sum(1 if c.lower() in 'aeiou' else 0                             
               for c in text)                                               
                                                                            
# get the signature object of the function                                                           
sig = inspect.signature(num_vowels)                                         

# show the argument named 'text'
print("text argument and its type: ", sig.parameters['text'])                                                      

# show the data type of 'text' argument using its annotation attribute
print("data type of text: ",sig.parameters['text'].annotation)                                           
                                                             
# print sig object
print("signature object: ", sig)                                                                         

# print the function's return type
print("return type of the function: ", sig.return_annotation)                                                       
                                                              
# print types of all the annotated parameters and return value
# note that we directly use the function name
print("annotations of the function: ", num_vowels.__annotations__)

text argument and its type:  text: str
data type of text:  <class 'str'>
signature object:  (text: str) -> int
return type of the function:  <class 'int'>
annotations of the function:  {'text': <class 'str'>, 'return': <class 'int'>}


In [97]:
# code example 9 for function information introspection

# fetch all class methods from Batch class using isfunction predicate
inspect.getmembers(batch.Batch, inspect.isfunction)

# fetch the signature object from the __init__() method of Batch class
init_sig = inspect.signature(batch.Batch.__init__)
# print the object as string
repr(init_sig)

# show the function arguments using its parameters attribute
init_sig.parameters

# show the default value of the iterables argument
repr(init_sig.parameters['iterables'].default)

'()'

### creating an object introspection tool
* inspect.getdoc(obj) get cleaned documents of the object
* getmembers() is the natural tool for getting the attributes and methods
* we developed our own version of getmembers
+ dump()
  + get all attributes using set(dir(object))
  + get only function attributes using callable(getattr(obj, attr_name)) in a lambda function as the predicate in filter
  + then get all non-function attributes by subtracting the function attributes using all_attr_names - method_names
  + organize attribute names and string representation using list comprehension (reprlib.repr was used to format the string)
+ full_sig(method)
  + concatenate method name and the string representation of the method signature
+ brief_doc(obj)
  + extract the first line of obj's dunder doc attribute if doc string is defined, otherwise, return an empty string
+ print_table function accepts a two-dimensional list as its 1st argument representing the data (rows_of_columns), and 2nd as a list of column headers (\*headers)
  + first check that the first row has the same number of elements as column headers
  + decompose the data and zip them with the column headers using chain() function
  rows_of_columns_with_header = itertools.chain(\[headers\], rows_of_columns)

In [98]:
# code example 10
import inspect
import itertools
import reprlib


def dump(obj):
    print("Type")
    print("====")
    print(type(obj))
    print()

    print("Documentation")
    print("=============")
    print(inspect.getdoc(obj))
    print()

    # get all attributes
    all_attr_names = set(dir(obj))
    
    # get all method attributes
    # note that callable(obj) returns True is obj is callable
    method_names = set(
        filter(lambda attr_name: callable(getattr(obj, attr_name)),
               all_attr_names))
    assert method_names <= all_attr_names
    attr_names = all_attr_names - method_names

    print("Attributes")
    print("==========")
    attr_names_and_values = [(name, reprlib.repr(getattr(obj, name)))
                             for name in attr_names]
    print_table(attr_names_and_values, "Name", "Value")
    print()

    print("Methods")
    print("=======")
    methods = (getattr(obj, method_name) for method_name in method_names)
    method_names_and_doc = sorted((full_sig(method), brief_doc(method))
                                  for method in methods)
    print_table(method_names_and_doc, "Name", "Description")
    print()


# get full signature of the method
def full_sig(method):
    try:
        return method.__name__ + str(inspect.signature(method))
    except ValueError:
        return method.__name__ + '(...)'


# get the brief document of the object
def brief_doc(obj):
    doc = obj.__doc__
    if doc is not None:
        lines = doc.splitlines()
        if len(lines) > 0:
            return lines[0]
    return ''


# rows_of_columns is a 2d list of method-names_and_doc
# heads is the list of all the remaining arguments, here is "Name" and "Description"
def print_table(rows_of_columns, *headers):
    num_columns = len(rows_of_columns[0])
    num_headers = len(headers)
    if len(headers) != num_columns:
        raise TypeError("Expected {} header arguments, "
                        "got {}".format(num_columns, num_headers))
    
    # convert row-based data to column-based data with column names
    # see code example 11 for details
    rows_of_columns_with_header = itertools.chain([headers], rows_of_columns)
    columns_of_rows = list(zip(*rows_of_columns_with_header))
    
    # find the max length of the elements in each column and use that to
    # define the column width
    column_widths = [max(map(len, column)) for column in columns_of_rows]
    
    # format each column using the column width. See example 12 for details
    column_specs = ('{{:{w}}}'.format(w=width) for width in column_widths)
    format_spec = ' '.join(column_specs)
    print(format_spec.format(*headers))
    rules = ('-' * width for width in column_widths)
    print(format_spec.format(*rules))
    for row in rows_of_columns:
        print(format_spec.format(*row))


In [99]:
# code example 11. How to chain column headers (1d list) with row-based data (2d list)
# and use zip to construct column-based data
headers = ["header1", "header2", "header3"]
a = chain([headers], [[4, 5, 6], [4, 5, 6]])
# print(list(a))
list(zip(*a))

[('header1', 4, 4), ('header2', 5, 5), ('header3', 6, 6)]

In [100]:
# code example 12
# we construcct format to define the column width using {"width"}
# then just put variables in .format()
# variable values will be printed out with the specified width

# define widths
widths = [10, 15, 20]

# construct column width format string as column_spec
column_spec = " ".join(('{{:{w}}}'.format(w=width) for width in widths))
print(column_spec)

# fill in the header values with the specified width
print(column_spec.format(*headers))

{:10} {:15} {:20}
header1    header2         header3             


In [101]:
def test_func():
    return 0
callable(test_func)

True

In [102]:

dump(7)

Type
====
<class 'int'>

Documentation
int([x]) -> integer
int(x, base=10) -> integer

Convert a number or string to an integer, or return 0 if no arguments
are given.  If x is a number, return x.__int__().  For floating point
numbers, this truncates towards zero.

If x is not a number or if base is given, then x must be a string,
bytes, or bytearray instance representing an integer literal in the
given base.  The literal can be preceded by '+' or '-' and be surrounded
by whitespace.  The base defaults to 10.  Valid bases are 0 and 2-36.
Base 0 means to interpret the base from the string as an integer literal.
>>> int('0b100', base=0)
4

Attributes
Name        Value                         
----------- ------------------------------
__doc__     "int([x]) -> ...', base=0)\n4"
real        7                             
numerator   7                             
imag        0                             
denominator 1                             

Methods
Name                             

#### Summary:
* inspect.getdoc() retrieves nicely formatted docstrings
* set Objects can be used to work with relationships between groups of objects
* clarity and readability are factors to consider when deciding between map and comprehensions
* a common docstring convention is that the first line is a summary
* format specifiers can be nested
* inspect gives you tools that can be combined to build introspection tools